In [2]:
from src.dataset.converter import single_text, double_text, double_text_double_label
from process import * 
from sklearn.model_selection import train_test_split

In [3]:
data_dir = './trainsample'
df = data_process('./trainsample/Train_Data.csv')

          negative      l_title       l_text     l_entity    l_kentity  \
count  4980.000000  4980.000000  4980.000000  4980.000000  4980.000000   
mean      0.555221    72.532530   219.405823     1.872490     0.983333   
std       0.496991    63.713498   446.521731     1.099766     1.159989   
min       0.000000     0.000000     5.000000     1.000000     0.000000   
50%       1.000000    56.000000   141.000000     2.000000     1.000000   
75%       1.000000   144.000000   181.250000     2.000000     2.000000   
95%       1.000000   156.000000   546.000000     4.000000     3.000000   
99%       1.000000   161.210000  2432.460000     5.000000     4.000000   
max       1.000000   201.000000  6872.000000    21.000000    18.000000   

           l_other  
count  4980.000000  
mean      0.889157  
std       1.077702  
min      -1.000000  
50%       1.000000  
75%       1.000000  
95%       3.000000  
99%       5.000000  
max      12.000000  


## Format1： 双输入文本+二分类任务
- text1：实体
- text2：title+text
- 其他伴随实体都用[O]进行替换

In [11]:

df1 = task_format1(df)
train, valid = train_test_split(df1, test_size= 0.2, random_state=1234)
train.to_csv('./trainsample/train.csv')
valid.to_csv('./trainsample/valid.csv')

double_text(train['id'], train['single_entity'], train['corpus'], train['label'], data_dir, 'train1')
double_text(valid['id'], valid['single_entity'], valid['corpus'], valid['label'], data_dir, 'valid1')

          negative      l_title       l_text     l_entity    l_kentity  \
count  4980.000000  4980.000000  4980.000000  4980.000000  4980.000000   
mean      0.555221    72.532530   219.405823     1.872490     0.983333   
std       0.496991    63.713498   446.521731     1.099766     1.159989   
min       0.000000     0.000000     5.000000     1.000000     0.000000   
50%       1.000000    56.000000   141.000000     2.000000     1.000000   
75%       1.000000   144.000000   181.250000     2.000000     2.000000   
95%       1.000000   156.000000   546.000000     4.000000     3.000000   
99%       1.000000   161.210000  2432.460000     5.000000     4.000000   
max       1.000000   201.000000  6872.000000    21.000000    18.000000   

           l_other  
count  4980.000000  
mean      0.889157  
std       1.077702  
min      -1.000000  
50%       1.000000  
75%       1.000000  
95%       3.000000  
99%       5.000000  
max      12.000000  


##  Format2: 单输入文本+二分类任务
- text：title+text
- 待预测实体用’[E]‘在实体的左右边界进行标记，其他伴随实体都用[O]进行替换

In [ ]:
df2 = task_format2(df)
train, valid = train_test_split(df2, test_size= 0.2, random_state=1234)
single_text(train['id'], train['corpus'], train['label'], data_dir, 'train2')
single_text(valid['id'], valid['corpus'], valid['label'], data_dir, 'valid2')


##  Format3：双输入文本+二分类任务
- text1：其他实体拼接
- text2：title+text
- 待预测实体用’[E]‘在实体的左右边界进行标记

In [ ]:
df3 = task_format3(df)
train, valid = train_test_split(df3, test_size= 0.2, random_state=1234)
double_text(train['id'], train['other_entity'], train['corpus'], train['label'], data_dir, 'train3')
double_text(valid['id'], valid['other_entity'], valid['corpus'], valid['label'], data_dir, 'valid3')

##  Format4：双输入文本+二分类任务+多任务
- text1：其他实体拼接
- text2：title+text
- 待预测实体用’[E]‘在实体的左右边界进行标记
- 任务1是实体负向，任务2是句子负向，用句子整体来辅助实体学习

In [21]:
df4 = task_format4(df)
train, valid = train_test_split(df4, test_size= 0.2, random_state=1234)
double_text_double_label(train['id'], train['other_entity'], train['corpus'], train['label1'], train['label2'], data_dir, 'train4')
double_text_double_label(valid['id'], valid['other_entity'], valid['corpus'], valid['label1'], train['label2'], data_dir, 'valid4')

## TAPT样本

In [4]:
test = data_process('./trainsample/Test_Data.csv')

           l_title       l_text     l_entity
count  4982.000000  4982.000000  4982.000000
mean     73.797270   197.142513     1.894420
std      64.569119   331.566611     1.070536
min       0.000000     0.000000     1.000000
50%      60.000000   141.000000     2.000000
75%     144.000000   179.000000     2.000000
95%     157.000000   526.950000     4.000000
99%     162.000000  1809.040000     6.000000
max     343.000000  5812.000000    21.000000


In [6]:
df = pd.concat([test.loc[:,['merge_text','entity']], df.loc[:,['merge_text','entity']]])

In [11]:
from process import task_whole_word_mlm
from src.dataset.chinese_ref import single_text_with_ref
df = task_whole_word_mlm(df)

In [13]:
single_text_with_ref(range(df.shape[0]), df['merge_text'].values, df['ref_ids'].values, './trainsample','train_mlm')